In [4]:
from IPython.utils.path import glob
import cv2
import urllib
import itertools
import numpy as np 
import pandas as pd
import seaborn as sns
import random,os,glob
from imutils import paths
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from urllib.request import urlopen

import warnings 
warnings.filterwarnings('ignore')

from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.layers import Conv2D,Flatten,MaxPooling2D,Dense,Dropout,SpatialDropout2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array, load_img,array_to_img


In [5]:
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
dir_path='/content/drive/MyDrive/Garbageclassification'

In [7]:
target_size=(224,224)
waste_labels={'cardboard':0,'glass':1,'metal':2,'paper':3,'plastic':4,'trash':5,}

In [8]:
def load_datasets(path):


  x=[]
  labels=[]

  image_paths=sorted(list(paths.list_images(path)))

  for image_path in image_paths: 
    img=cv2.imread(image_path)
    img=cv2.resize(img, target_size)
    x.append(img)
    label=image_path.split(os.path.sep)[-2]
    labels.append(waste_labels[label])

  x, labels=shuffle(x,labels,random_state=42)
  print(f"X boyutu :{np.array(x).shape}")
  print(f"label sınıf sayısı: {len(np.unique(labels))} Gözlem sayısı : {len(labels)}")
  return x ,labels

In [ ]:
x, labels=load_datasets(dir_path)

In [ ]:
input_shape=(np.array(x[0]).shape[1],np.array(x[0]).shape[1],3)
 #ilksatır
print(input_shape)

In [ ]:
def visualize_img(image_batch,label_batch):
  plt.figure(figsize=(10,10))
  for n in range(10):
   ax=plt.subplot(5,5,n+1)
   plt.imshow(image_batch[n])
   plt.title(np.array(list(waste_labels.keys()))[to_categorical(labels,num_classes=6)[n]==1][0].title())
   plt.axis('off')

In [ ]:
visualize_img(x,labels)

In [ ]:
train=ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         validation_split=0.1,
                         rescale=1./255,
                         shear_range=0.1,
                         zoom_range=0.1,
                         width_shift_range=0.1,
                         height_shift_range=0.1)
test=ImageDataGenerator(rescale=1/255,
                        validation_split=0.1)

In [ ]:
train_generator=train.flow_from_directory(directory=dir_path,
                                          target_size=(target_size),
                                          class_mode='categorical',
                                          subset='training')

In [ ]:
test_generator=test.flow_from_directory(directory=dir_path,
                                          batch_size=251,
                                          class_mode='categorical',
                                          subset='validation')

In [ ]:
model=Sequential()
model.add(Conv2D(filters=32,kernel_size=(3,3),padding='same',input_shape=(input_shape),activation='relu'))
model.add(MaxPooling2D(pool_size=2,strides=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2,strides=(2,2)))

model.add(Conv2D(filters=32,kernel_size=(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2,strides=(2,2)))

model.add(Flatten())

model.add(Dense(units=64,activation='relu'))
model.add(Dropout(rate=0.2))

model.add(Dense(units=32,activation='relu'))
model.add(Dropout(rate=0.2))

model.add(Dense(units=6,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),"acc"])

callbacks=[EarlyStopping(monitor='val_loss',patience=50,verbose=1,mode="min"),
           ModelCheckpoint(filepath='mymodel.h5',monitor='val_loss',mode='min',save_best_only=True,save_weights_only=False,verbose=1)]

In [ ]:
history=model.fit_generator(generator=train_generator,
                            epochs=100,
                            validation_data=test_generator,
                            callbacks=callbacks,
                            workers=4,
                            steps_per_epoch=2276//32,
                            validation_steps=251//32
                            )

In [ ]:
plt.figure(figsize=(30,5))

plt.subplot(1,2,1)
plt.plot(history.history['acc'],label='training Accuracy')
plt.plot(history.history['val_acc'], label='validation Accuracy')
plt.legend(loc='lower right')
plt.xlabel('epoch',fontsize=16)
plt.ylabel('accuracy',fontsize=16)
plt.ylim([min(plt.ylim()),1])
plt.title('eğitim ve test başarım Accuracy',fontsize=16)

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.legend(loc='upper right')
plt.xlabel('epoch',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.ylim([0,max(plt.ylim())])
plt.title('eğitim ve test kayıp grafiği',fontsize=16)
plt.show()

In [ ]:
loss,precision,recall,acc=model.evaluate(test_generator,batch_size=32)

print("/ntest accuracy : %.1%%" % (100.0*acc))
print("/ntest loss: %.1%%" % (100.0*loss))
print("/ntest precision: %.1%%" % (100.0*precision))
print("/ntest recall: %.1%%" % (100.0*recall ))


In [ ]:
x_test,y_test=test_generator.next()
y_pred=model.predict(x_test)

y_pred=np.argmax(y_pred,axis=1)
y_test=np.argmax(y_test,axis=1)

target_names=list(waste_labels.keys())

print(classification_report(y_test,y_pred,target_names=target_names))